
---

### Reminder: This 📘 `.NET Interactive` notebook needs to be run from VS Code with [these prerequisites](../PREREQS.md).

#### How to use this notebook: 

* Just read the text and scroll along until you run into code blocks.
* Code blocks have computer code inside them — hover over the block and you can run the code.
* Run the code by hitting the ▶️ "play" button to the left. If the code runs you'll see a ✔️. If not, you'll get a ❌.
* The output and status of the code block will appear just below itself — you need to scroll down further to see it.
* Sometimes a code block will ask you for input in a hard-to-notice dialog box 👆 at the top of your notebook window. 

---

# Recipe VI: 🔥 Kernel Chain Reactions
## 🧑‍🍳 Masterfully cook chain 🧂🧂🧂🔥 reactions from multiple semantic functions

When people ask me why Semantic Kernel has its slightly odd name, the simple and truthful answer is that the initial conceiver of SK, Deputy CTO of Microsoft Sam Schillace, has been a long admirer of the simplicity of the UNIX kernel. UNIX was a radical departure in operating systems for its simplicity and flexibility — especially when it came to a concept called "pipes." UNIX commands were easy to string together into a chain reaction like:

`% do-something | do-more-stuff | do-even-more-stuff >> drop-it-all-in-here.txt`

SK lets you do the same thing with the ability to compose functions together as in a chain of functions that take an input that produces an output, and they are connected end-to-end. For example, let's imagine we want to take a bunch of feedback items from customers and understand their painpoints. We can use the `DesignThinkingSkill.Empathize` to give that a go.

## Step 1. Instantiate a 🔥 kernel

In [1]:
#r "nuget: Microsoft.SemanticKernel, 0.9.61.1-preview"

#!import ../config/Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.KernelExtensions;
using System.IO;
using Microsoft.SemanticKernel.Configuration;
using Microsoft.SemanticKernel.SemanticFunctions;

IKernel kernel = Microsoft.SemanticKernel.Kernel.Builder.Build();

// Grab the locally stored credentials from the settings.json file. Name the "backend" as "davinci" — assuming that you're using one of the davinci completion models. 

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    kernel.Config.AddAzureOpenAITextCompletion("davinci", model, azureEndpoint, apiKey);
else
    kernel.Config.AddOpenAITextCompletion("davinci", model, apiKey, orgId);

Installed Packages Microsoft.SemanticKernel, 0.9.61.1-preview

## Step 2: Load up a 🧂 skill and warm up the 🔥 kernel

Empathy is the ability to understand and share the feelings of another. It is a key skill for designers, as it allows them to understand the needs of the people they are designing for. Empathy is also a key skill for developers, as it allows them to address the needs of the people they are building for.

One quick way to get a sense for one's users and customers is to analyze data that's readily available: usually in the form of customer support logs or notes from sales team about difficult customer interactions that have recently transpired. A simple SK skill to analyze that information would read as:

In [2]:
using System.Text.Json;
using System.Text.Json.Serialization;

// Load the Skills Directory
var skillsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "skills");

// Load the EmpathizeSkill from the Skills Directory
var skillDT = kernel.ImportSemanticSkillFromDirectory(skillsDirectory, "DesignThinkingSkill");

// The default input variable 
var input = @"
Customer 1: The power button on my phone is broken. The warranty is still valid.
Customer 2: My display stopped working.
Customer 3: The customer service rep didn't answer my email.
Customer 4: Every time I call customer support I get no answer.
Customer 5: The display screen cracked and it's still under warranty.
Customer 6: My power button fell off the phone. That's ridiculous.
Customer 7: I'm so frustrated with this company.
Customer 8: When I use the power button too much, it stops working.
";

var empathyResult = await kernel.RunAsync(input, skillDT["Empathize"]);

Console.WriteLine(empathyResult);



[
    { "sentiment": "Frustrated", "summary": "Poor customer service"},
    { "sentiment": "Dissatisfied", "summary": "Products are not functioning properly"},
    { "sentiment": "Angry", "summary": "Products are still under warranty but not functioning properly"}
]


This is commonly known as the "Empathize" phase of Stanford University's approach to "Design Thinking" with the other phases being: Define, Ideate, Prototype, Test. 

![](../assets/empathize.png) 

So in design thinking parlance, we have done the important first step of "Empathize" by understanding our customers' painpoints.

## Step 3. 🧂🔥 Turn the empathy insights into problem definitions

We can take this analysis and feed it into a problem-definer skill we have in the `DesignThinkingSkill` toolkit called `Define`:

In [3]:

// Load the Skills Directory
var skillsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "skills");

// Load the EmpathizeSkill from the Skills Directory
var skillDT = kernel.ImportSemanticSkillFromDirectory(skillsDirectory, "DesignThinkingSkill");

var defineResult = await kernel.RunAsync(empathyResult.ToString(), skillDT["Define"]);

Console.WriteLine(defineResult);



[
    { "title": "Inadequate Support", "problem": "Customers are not receiving the help they need in a timely manner.", "source":"Lack of resources or staff to provide timely assistance."},
    { "title": "Defective Products", "problem": "Products are not functioning properly or are not up to the expected standards.", "source":"Poor quality control or inadequate testing."},
    { "title": "Misleading Warranty", "problem": "Products are still under warranty but not functioning properly.", "source":"Misleading advertising or lack of clear warranty terms."},
    { "title": "Unclear Policies", "problem": "Customers are not aware of the policies or are confused by them.", "source":"Lack of clear communication or lack of customer education."},
    { "title": "Unresponsive Staff", "problem": "Customer service staff are not responding to customer inquiries.", "source":"Lack of training or resources to handle customer inquiries."}
]


## Step 4. 🧂🔥 Brainstorm ideas to address the problems

And if we were in a mood to attempt to solve their problem, we could use the `DesignThinkingSkill.Ideate` function to brainstorm ideas to address the problems as easy versus difficult:

In [4]:

// Load the Skills Directory
var skillsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "skills");

// Load the EmpathizeSkill from the Skills Directory
var skillDT = kernel.ImportSemanticSkillFromDirectory(skillsDirectory, "DesignThinkingSkill");

var ideateResult = await kernel.RunAsync(defineResult.ToString(), skillDT["Ideate"]);

//Console.WriteLine(ideateResult);
display(ideateResult.ToString());




| Lower-hanging fruit | Higher-hanging fruit |
| :--- | :--- |
| Increase customer service staff | Develop a customer loyalty program |
| Develop a customer feedback system | Create a mobile app for customers |
| Automate customer service inquiries | Implement a customer service chatbot |
| Develop a customer service knowledge base | Implement a customer service CRM system |
| Improve customer service training | Invest in customer service analytics |
| Improve customer service processes | Create a customer service portal |
| Implement a customer service ticketing system | Develop a customer service feedback system |
| Create customer service tutorials | Invest in customer service technology |
| Invest in customer service automation | Create a customer service reward program |

## Step 6. 🧂🧂🧂🔥 Do it all again, but as a chain reaction 

Each step unlocks more power. But what if we could just chain these all together instead of just incrementally process them. That would set a chain reaction in motion from customer support logs all the way to ideas of how to address their concerns. Let's build that:

In [5]:
#r "nuget: Markdig.Signed, 0.31.0"
using Microsoft.AspNetCore.Html;
using Markdig;
using System.Text.Json;
using System.Text.Json.Serialization;

// Load the Skills Directory
var skillsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "skills");

// Load the EmpathizeSkill from the Skills Directory
var skillDT = kernel.ImportSemanticSkillFromDirectory(skillsDirectory, "DesignThinkingSkill");

var input = @"
Customer 1: The power button on my phone is broken. The warranty is still valid.
Customer 2: My display stopped working.
Customer 3: The customer service rep didn't answer my email.
Customer 4: Every time I call customer support I get no answer.
Customer 5: The display screen cracked and it's still under warranty.
Customer 6: My power button fell off the phone. That's ridiculous.
Customer 7: I'm so frustrated with this company.
Customer 8: When I use the power button too much, it stops working.
";

var myResult = await kernel.RunAsync(input, skillDT["Empathize"], skillDT["Define"], skillDT["Ideate"]);

var myPipeline = new MarkdownPipelineBuilder().UseAdvancedExtensions().Build();
var myResult2HTML = Markdown.ToHtml("# Ideate: 'Design Thinking' directions generated from customer empathy (Empathize) and problem definition (Define)\n\n" + myResult.ToString(), myPipeline);
var myHTMLContent = new HtmlContentBuilder();

myHTMLContent.AppendHtml(myResult2HTML);
myHTMLContent

Installed Packages Markdig.Signed, 0.31.0

Lower-hanging fruit,Higher-hanging fruit
Automated customer service chatbot,AI-driven customer service
Automated warranty repair process,Augmented reality support
Mobile app for device management,Cloud-based customer service
Online tutorials and videos,Automated customer service analytics
Online forums for customer support,Voice-activated customer service
Improved user manuals,Improved customer service tracking
In-store support kiosks,Automated customer service scheduling
Remote technical support,Automated customer service escalation
Online customer service chat,Self-service customer service portal
Improved customer service training,Proactive customer service notifications


Neat, huh? We just connected each of the functions together so the first input goes into the next output, goes into a new input, and sends out a new output, etc.

# ⏭️ Next Steps

Run through more advanced examples in the notebooks that are available in our GitHub repo at [https://aka.ms/sk/repo](https://aka.ms/sk/repo).

[Ready for a bonus round? Got chat?](../e7-bonus-chat/notebook.ipynb)

Or stay a longer while and alter the customer support comments to get suggestions for your own product or services.